CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect

  Obtaining dependency information for alibi-detect from https://files.pythonhosted.org/packages/ed/2a/9e11bfee0cf54c0ea78243cb0559d604878f6d58d0ac6227d11e7007decc/alibi_detect-0.11.4-py3-none-any.whl.metadata
  Using cached Pillow-9.5.0-cp38-cp38-macosx_10_10_x86_64.whl (3.4 MB)
  Obtaining dependency information for opencv-python<5.0.0,>=3.2.0 from https://files.pythonhosted.org/packages/05/58/7ee92b21cb98689cbe28c69e3cf8ee51f261bfb6bc904ae578736d22d2e7/opencv_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl.metadata
  Obtaining dependency information for scikit-image!=0.17.1,<0.22,>=0.14.2 from https://files.pythonhosted.org/packages/35/e4/d5d1574d09f30a4df757edf4213ce8e764aebe0f1642475cf384f9fa33bb/scikit_image-0.21.0-cp38-cp38-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.0.0 from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb03bccf0a9c2fec1/transformers-4.33.3-py3-none-any.whl.metadata
 

In [31]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

import torch
import pytorch_tabular
import math
from sklearn.metrics import mean_squared_error, r2_score

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [16]:
df = pd.read_csv('hdb_price_prediction.csv')

dir = 'saved_models/part_b1'
tabular_model = pytorch_tabular.tabular_model.TabularModel.load_model(dir)

df_2022 = df[df['year'] == 2022]
pred_df = tabular_model.predict(df_2022)

target = pred_df['resale_price']
prediction = pred_df['resale_price_prediction']
print(f"R^2: {r2_score(target, prediction)}")

2023-10-04 18:29:21,687 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-10-04 18:29:21,689 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

R^2: 0.43884728720843


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [18]:
# TODO: Enter your code here
df_2023 = df[df['year'] == 2023]
pred_df = tabular_model.predict(df_2023)

target = pred_df['resale_price']
prediction = pred_df['resale_price_prediction']
print(f"R^2: {r2_score(target, prediction)}")

Output()

R^2: 0.16212593972873102


> Did model degradation occur for the deep learning model?


Yes, the $R^2$ value has decreased



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [73]:
categorical_columns = ['month', 'town', 'flat_model_type', 'storey_range']
continuous_columns = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
target_columns = ['resale_price']

X = df[categorical_columns + continuous_columns]
y = df[target_columns]
feature_names = X.columns.values
category_map = {}
for i in range(len(X.columns)):
    if X.columns[i] in categorical_columns:
        category_map[i] = df[X.columns[i]].unique().tolist()

df_train = df[df['year'] <= 2019]
df_2023 = df[df['year'] == 2023]

n_ref = 1000
n_test = 1000

df_train = df_train[:n_ref]
df_2023 = df_2023[:n_test]

X_ref = df_train[categorical_columns + continuous_columns].values
y_ref = df_train[target_columns].values

X_test = df_2023[categorical_columns + continuous_columns].values
y_test = df_2023[target_columns].values

categories_per_feature = {f: None for f in list(category_map.keys())}
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)

In [81]:
preds = cd.predict(X_test)
labels = ['No', 'Yes']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))
threshold = preds['data']['threshold']
print("Threshold:", threshold)

drifted_columns = []
fpreds = cd.predict(X_test, drift_type='feature')
print("The following features have drifted:")
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

    if is_drift:
        drifted_columns.append(fname)

Drift? Yes
Threshold: 0.005
The following features have drifted:
month -- Drift? No -- Chi2 0.000 -- p-value 1.000
town -- Drift? Yes -- Chi2 667.474 -- p-value 0.000
flat_model_type -- Drift? Yes -- Chi2 77.586 -- p-value 0.000
storey_range -- Drift? Yes -- Chi2 38.800 -- p-value 0.001
dist_to_nearest_stn -- Drift? No -- K-S 0.055 -- p-value 0.094
dist_to_dhoby -- Drift? Yes -- K-S 0.218 -- p-value 0.000
degree_centrality -- Drift? No -- K-S 0.029 -- p-value 0.783
eigenvector_centrality -- Drift? Yes -- K-S 0.195 -- p-value 0.000
remaining_lease_years -- Drift? Yes -- K-S 0.271 -- p-value 0.000
floor_area_sqm -- Drift? Yes -- K-S 0.134 -- p-value 0.000


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


Concept drift

> From your analysis via TabularDrift, which features contribute to this shift?


In [77]:
drifted_columns

['town',
 'flat_model_type',
 'storey_range',
 'dist_to_dhoby',
 'eigenvector_centrality',
 'remaining_lease_years',
 'floor_area_sqm']

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


Keep updating the model by training it on new data

In [83]:
# TODO: Enter your code here
tabular_model.fit(train=df[df['year'] <= 2022])

Global seed set to 42


2023-10-04 19:14:41,151 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-04 19:14:41,172 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-04 19:14:41,428 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-04 19:14:41,467 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-10-04 19:14:41,52

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at /Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/.lr_find_999ae892-c42f-4852-bedc-6cf5bad23b38.ckpt
Restored all states from the checkpoint file at /Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/.lr_find_999ae892-c42f-4852-be

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-04 19:16:39,160 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-04 19:16:39,161 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [84]:
test = df[df['year'] == 2023]

result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)

target = pred_df['resale_price']
prediction = pred_df['resale_price_prediction']
print(f"R^2: {r2_score(target, prediction)}")

Output()

/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/core/m
odule.py:493: UserWarning: You called `self.log('test_loss', ..., logger=True)` but have no logger configured. You 
can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/core/m
odule.py:493: UserWarning: You called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger 
configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/tohhongxiang/Desktop/programming/CZ4042-Assignment/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       13991294976.0       │
│  test_mean_squared_error  │       13991294976.0       │
└───────────────────────────┴───────────────────────────┘

Output()

R^2: 0.5254130626533


By training the model again on more recent data, we can now see that the $R^2$ value has improved

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |